In [5]:
from dataset.data_preprocessing import *
from Preprocessing_Functions2 import *
from Training_Helper_Functions import *
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import os
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, roc_auc_score
import matplotlib.pyplot as plt



Using cuda


In [6]:
load_and_preprocess_data("dataset/raw_data.csv") #? convert raw data to OHE and encoded version
data = pd.read_csv("dataset/processed_data_encoded.csv")

#? Split training and holdout/test set
trainVal, testing = train_test_split(data, test_size=0.1, random_state=42, stratify=data["DR"])

#? Split training and validation set
training, validation = train_test_split(trainVal, test_size=1/9, random_state=42, stratify=trainVal["DR"])

def CV_generator(training:pd.DataFrame, validation:pd.DataFrame, OD_majority=None, OD_minority=None,synthesizer = "TVAE", epochs = 1000, n_synthetic_data=10000, scaler=StandardScaler()):
    cont_cols = ['Age', 'UAlb', 'Ucr', 'UACR', 'TC', 'TG', 'TCTG', 'LDLC', 'HDLC', 
             'Scr', 'BUN', 'FPG', 'HbA1c', 'Height', 'Weight', 'BMI', 'Duration']
    # Use the original encoded single column name here
    cat_cols = ['Gender', 'Community'] 
    y_col = 'DR'
    
    X = training.drop(columns=["BMI", "TCTG"])
    validation = validation.drop(columns=["BMI", "TCTG"])

    #* OUTLIER DETECTION
    X_train_processed = Outlier_Removal(X, 
                                    OD_majority=OD_majority,
                                    OD_minority=OD_minority,
    )
    # #* OVERSAMPLING & SYNTHETIC DATA GENERATION
    print("Before oversampling & synthetic data:", X_train_processed[["DR"]].value_counts())
    #* Smote but we don't use it for training
    # _train_processed = apply_smotenc_oversampling(X_train_processed)
    X_train_processed = Synthetic_Data_Generator2(X_train_processed, "", synthesizer=synthesizer, epochs=epochs, batch_size=256, n_synthetic_data=n_synthetic_data)
    print("After oversampling & synthetic data:", X_train_processed[["DR"]].value_counts())

    
    #* Calculate BMI, TCTG & ENCODING
    X_train_processed, validation = get_bmi(X_train_processed, validation)
    X_train_processed, validation = get_TCTG(X_train_processed, validation)
    X_train_processed, validation = apply_one_hot_encoding(X_train_processed, validation)
    #* Scaler
    X_train_processed[cont_cols] = scaler.fit_transform(X_train_processed[cont_cols])
    validation[cont_cols] = scaler.transform(validation[cont_cols])
    # Append processed data (excluding the target column 'DR')

    # Save to CSV with fold number
    X_train_processed.to_csv(f"./DATA/training80.csv", index=False)
    validation.to_csv(f"./DATA/val10.csv", index=False)
    
    return

def final_generator(training:pd.DataFrame, test:pd.DataFrame,
OD_majority=None, OD_minority=None,synthesizer = "TVAE", epochs = 1000, n_synthetic_data=10000, scaler=StandardScaler()):
    cont_cols = ['Age', 'UAlb', 'Ucr', 'UACR', 'TC', 'TG', 'TCTG', 'LDLC', 'HDLC', 
             'Scr', 'BUN', 'FPG', 'HbA1c', 'Height', 'Weight', 'BMI', 'Duration']
    # Use the original encoded single column name here
    cat_cols = ['Gender', 'Community'] 
    y_col = 'DR'
    
    X = training.drop(columns=["BMI", "TCTG"])
    test = test.drop(columns=["BMI", "TCTG"])

    #* OUTLIER DETECTION
    X_train_processed = Outlier_Removal(X, 
                                    OD_majority=OD_majority,
                                    OD_minority=OD_minority,
    )
    # #* OVERSAMPLING & SYNTHETIC DATA GENERATION
    print("Before oversampling & synthetic data:", X_train_processed[["DR"]].value_counts())
    #* Smote but we don't use it for training
    # _train_processed = apply_smotenc_oversampling(X_train_processed)
    X_train_processed = Synthetic_Data_Generator2(X_train_processed, "", synthesizer=synthesizer, epochs=epochs, batch_size=256, n_synthetic_data=n_synthetic_data)
    print("After oversampling & synthetic data:", X_train_processed[["DR"]].value_counts())

    
    #* Calculate BMI, TCTG & ENCODING
    X_train_processed, test = get_bmi(X_train_processed, test)
    X_train_processed, test = get_TCTG(X_train_processed, test)
    X_train_processed, test = apply_one_hot_encoding(X_train_processed, test)
    #* Scaler
    X_train_processed[cont_cols] = scaler.fit_transform(X_train_processed[cont_cols])
    test[cont_cols] = scaler.transform(test[cont_cols])
    # Append processed data (excluding the target column 'DR')

    # Save to CSV with fold number
    X_train_processed.to_csv(f"./DATA/training90.csv", index=False)
    test.to_csv(f"./DATA/test10.csv", index=False)
    
    return

Processed (one-hot) data saved to processed_data_OHE.csv
Processed (label-encoded) data saved to processed_data_encoded.csv


In [ ]:
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# CV_generator(training, validation, OD_majority=None, OD_minority=None,synthesizer = "TVAE", epochs = 1000, n_synthetic_data=10000, scaler=StandardScaler())

CV_generator(training, validation,
            OD_majority = IQRDetector(factor=1.2),
            OD_minority = IQRDetector(factor=1.8),
            synthesizer = "TVAE",
            epochs = 1000,
            n_synthetic_data = 10000,
            scaler=StandardScaler())

# final_generator(training, testing, OD_majority=None, OD_minority=None,synthesizer = "TVAE", epochs = 1000, n_synthetic_data=10000, scaler=StandardScaler())

final_generator(training, testing,
                OD_majority = IQRDetector(factor=1.2),
                OD_minority = IQRDetector(factor=1.8),
                synthesizer = "TVAE",
                epochs = 1000,
                n_synthetic_data = 10000,
                scaler=StandardScaler())

)
training80 = pd.read_csv("./DATA/training80.csv")
val10 = pd.read_csv("./DATA/val10.csv")
training90 = pd.read_csv("./DATA/training90.csv")
test10 = pd.read_csv("./DATA/test10.csv")

training80_x = training80.drop(columns=["DR"])
training80_y = training80[["DR"]]
val10_x = val10.drop(columns=["DR"]) 
val10_y = val10[["DR"]]

training90_x = training90.drop(columns=["DR"])
training90_y = training90[["DR"]]
test10_x = test10.drop(columns=["DR"]) 
test10_y = test10[["DR"]]

train80_loader, val10_loader = fold_to_dataloader_tensor(training80_x, training80_y, val10_x, val10_y, batch_size=256)

training90_loader, test10_loader = fold_to_dataloader_tensor(training90_x, training90_y, test10_x, test10_y, batch_size=256)

Original class distribution: DR
0.0    4588
1.0     516
Name: count, dtype: int64
Before oversampling & synthetic data: DR 
0.0    4588
1.0     516
Name: count, dtype: int64
Fitting synthesizer...


d:\GitHub repos\ADL2\.venv\Lib\site-packages\sdv\single_table\base.py:105: UserWarning: We strongly recommend saving the metadata using 'save_to_json' for replicability in future SDV versions.
  warnings.warn(
  File "d:\GitHub repos\ADL2\.venv\Lib\site-packages\joblib\externals\loky\backend\context.py", line 257, in _count_physical_cores
    cpu_info = subprocess.run(
               ^^^^^^^^^^^^^^^
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.11_3.11.2544.0_x64__qbz5n2kfra8p0\Lib\subprocess.py", line 548, in run
    with Popen(*popenargs, **kwargs) as process:
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.11_3.11.2544.0_x64__qbz5n2kfra8p0\Lib\subprocess.py", line 1026, in __init__
    self._execute_child(args, executable, preexec_fn, close_fds,
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.11_3.11.2544.0_x64__qbz5n2kfra8p0\Lib\subprocess.py", line 1538, in _execute_child
  

Generating synthetic samples per class based on distribution...


Sampling conditions: 100%|██████████| 1010/1010 [00:03<00:00, 255.08it/s]


Final synthetic class distribution:
DR
0.0    8989
1.0    1010
Name: count, dtype: int64
Generating report ...

(1/2) Evaluating Column Shapes: |██████████| 18/18 [00:03<00:00,  4.75it/s]|
Column Shapes Score: 90.59%

(2/2) Evaluating Column Pair Trends: |██████████| 153/153 [00:01<00:00, 100.35it/s]|
Column Pair Trends Score: 86.74%

Overall Score (Average): 88.67%

After oversampling & synthetic data: DR 
0.0    13577
1.0     1526
Name: count, dtype: int64
Original class distribution: DR
0.0    4588
1.0     516
Name: count, dtype: int64
Before oversampling & synthetic data: DR 
0.0    4588
1.0     516
Name: count, dtype: int64
Fitting synthesizer...


d:\GitHub repos\ADL2\.venv\Lib\site-packages\sdv\single_table\base.py:105: UserWarning: We strongly recommend saving the metadata using 'save_to_json' for replicability in future SDV versions.
  warnings.warn(
Loss: -11.023: 100%|██████████| 1000/1000 [14:28<00:00,  1.15it/s]


Generating synthetic samples per class based on distribution...


Sampling conditions: 100%|██████████| 1010/1010 [00:03<00:00, 268.64it/s]


Final synthetic class distribution:
DR
0.0    8989
1.0    1010
Name: count, dtype: int64
Generating report ...

(1/2) Evaluating Column Shapes: |██████████| 18/18 [00:02<00:00,  6.56it/s]|
Column Shapes Score: 91.3%

(2/2) Evaluating Column Pair Trends: |██████████| 153/153 [00:01<00:00, 95.21it/s]| 
Column Pair Trends Score: 86.96%

Overall Score (Average): 89.13%

After oversampling & synthetic data: DR 
0.0    13577
1.0     1526
Name: count, dtype: int64


RuntimeError: view size is not compatible with input tensor's size and stride (at least one dimension spans across two contiguous subspaces). Use .reshape(...) instead.

In [ ]:
import pickle
class Ivan_NN(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.input_fc = nn.Linear(input_dim, 2180)
        self.input_bn = nn.BatchNorm1d(2180)
        self.drop = nn.Dropout(0.3)

        self.block1 = nn.Sequential(
            nn.Linear(2180, 888),
            nn.BatchNorm1d(888),
            nn.LeakyReLU(),
            nn.Dropout(0.3)
        )

        self.block2 = nn.Sequential(
            nn.Linear(888, 1122),
            nn.BatchNorm1d(1122),
            nn.LeakyReLU(),
            nn.Dropout(0.3)
        )

        self.block3 = nn.Sequential(
            nn.Linear(1122, 624),
            nn.BatchNorm1d(624),
            nn.LeakyReLU(),
            nn.Dropout(0.3)
        )

        self.block4 = nn.Sequential(
            nn.Linear(624, 1080),
            nn.BatchNorm1d(1080),
            nn.LeakyReLU(),
            nn.Dropout(0.3)
        )

        self.block5 = nn.Sequential(
            nn.Linear(1080, 512),
            nn.BatchNorm1d(512),
            nn.LeakyReLU(),
            nn.Dropout(0.3)
        )

        self.block6 = nn.Sequential(
            nn.Linear(512, 128),
            nn.BatchNorm1d(128),  # ← Fixed this
            nn.LeakyReLU(),
            nn.Dropout(0.3)
        )

        self.block7 = nn.Sequential(
            nn.Linear(128, 36),
            nn.BatchNorm1d(36),
            nn.LeakyReLU(),
            nn.Dropout(0.2)
        )

        self.block8 = nn.Sequential(
            nn.Linear(36, 64),
            nn.BatchNorm1d(64),
            nn.Sigmoid(),  # or LeakyReLU, up to you
            nn.Dropout(0.1)
        )

        self.output = nn.Linear(64, 1)

        # Skip connection projectors
        self.skip1_proj = nn.Sequential(nn.Linear(2180, 1122))
        self.skip2_proj = nn.Sequential(nn.Linear(1122, 128))
        self.skip3_proj = nn.Sequential(nn.Linear(128, 64))

    def forward(self, x):
        x = self.input_fc(x)
        x = self.input_bn(x)
        x = torch.nn.functional.leaky_relu(x)
        x = self.drop(x)

        x1 = self.block1(x)
        x2 = self.block2(x1)

        skip1 = self.skip1_proj(x)
        x2 = x2 + skip1

        x3 = self.block3(x2)
        x4 = self.block4(x3)
        x5 = self.block5(x4)

        x6 = self.block6(x5)

        skip2 = self.skip2_proj(x2)
        x6 = x6 + skip2

        x7 = self.block7(x6)
        x8 = self.block8(x7)

        skip3 = self.skip3_proj(x6)
        x8 = x8 + skip3

        out = self.output(x8)
        return out

def train_model(model:Ivan_NN, nFeatures:int, train80_loader, val10_loader, epochs=10, learningRate = 0.0001, threshold:float = 0.5, patience = 30, device=None):
    # Use GPU if available
    device = device or ("cuda" if torch.cuda.is_available() else "cpu")

    theMODEL = model(nFeatures)
    theMODEL.to(device)
    print(theMODEL)

    posWeight = torch.tensor(10, dtype=torch.float32).to("cpu")
    criterion = nn.BCEWithLogitsLoss(pos_weight=posWeight)
    optimizer = optim.Adam(theMODEL.parameters() ,lr=learningRate, weight_decay=1e-4)

    train_losses = []
    val_losses = []
    train_accuracies = []
    val_accuracies= []
    val_precisions = []
    val_recalls = []
    val_f1s = []
    best_val_loss = float("inf")
    best_val_f1 = .0


    for epoch in range(epochs):
        theMODEL.train()
        running_loss = 0.0
        correct = 0
        total = 0

        for inputs, targets in train80_loader:
            optimizer.zero_grad()
            inputs, targets = inputs.to(device), targets.squeeze().to(device)

            # Forward pass
            outputs = theMODEL(inputs).squeeze()
            loss = criterion(outputs, targets)
            # Backward and optimize
            # torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=.5)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()# * inputs.size(0)

            # preds = torch.round(torch.sigmoid(outputs))
            preds = (torch.sigmoid(outputs) >= threshold).float()

            correct += (preds == targets).sum().item()
            total += targets.size(0)

        train_loss = running_loss / total
        train_acc = correct / total
        

        # Validation
        theMODEL.eval()
        val_loss = 0.0
        val_correct = 0
        val_total = 0

        with torch.no_grad():
            for inputs, targets in val10_loader:
                inputs, targets = inputs.to(device), targets.squeeze().to(device)
                outputs = theMODEL(inputs).squeeze()
                loss = criterion(outputs, targets)
                val_loss += loss.item() # * inputs.size(0)

                if pd.isna(loss.item()):
                    print( "Val Loss:" ,loss)
                    in_rows = torch.isnan(inputs).any(axis=1)
                    out_rows = torch.isnan(outputs).any(axis=-1)
                    targets_rows = torch.isnan(targets).any(axis=-1)
                    print(inputs[in_rows], outputs[out_rows], targets[targets_rows],sep="\n")
                    return
                
                # preds = torch.round(torch.sigmoid(outputs))
                preds = (torch.sigmoid(outputs) >= threshold).float()

                val_correct += (preds == targets).sum().item()
                val_total += targets.size(0)

                f1Score = f1_score(targets, preds)
                precision = precision_score(targets,preds,zero_division=.0)
                recall = recall_score(targets, preds)
                accuracy = accuracy_score(targets,preds)


        if val_total == 0:
            print("Empty validation set!")
            return

        val_loss /= val_total
        val_acc = val_correct / val_total

        train_losses.append(train_loss)
        val_losses.append(val_loss)
        train_accuracies.append(train_acc)
        val_accuracies.append(val_acc)
        val_precisions.append(precision)
        val_recalls.append(recall)
        val_f1s.append(f1Score)

        print(f"Epoch [{epoch+1}/{epochs}]".ljust(16),
              f"Train Loss: {train_loss:.6f} Acc: {train_acc:.6f} | "
            #   f"Val Loss: {val_loss:.6f} Acc: {val_acc:.6f}"
              f"Val Loss: {val_loss:.6f} Acc: {accuracy:.2f}% Precision: {precision:.2f} Recall: {recall:.2f} F1: {f1Score:.2f}")
        
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_loss_model = {
                "epoch": epoch+1,
                "stateDict": theMODEL.state_dict(),
            }
            print("Best loss model saved!")
        if f1Score > best_val_f1:
            best_val_f1 = f1Score
            best_f1_model = {
                "epoch": epoch+1,
                "stateDict": theMODEL.state_dict(),
            }
            print("Best f1 model saved!")
        elif val_loss > best_val_loss and epoch > best_loss_model["epoch"] + patience:
            print(f"Early stopping... Current Val Loss:{val_loss}  vs  Best Val Loss: {best_val_loss}")
            break

    plt.figure(figsize=(8, 5))
    plt.plot(train_losses, label='Train Loss', marker='^')
    plt.plot(val_losses, label='Val Loss', marker='x')
    plt.title("Training vs. Val Loss")
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()

    plt.figure(figsize=(8, 5))
    plt.plot(train_accuracies, label='Train Acc', marker='^')
    plt.plot(val_accuracies, label='Val Acc', marker='x')
    plt.title("Training vs. Val Acc")
    plt.xlabel("Epoch")
    plt.ylabel("Accuracy")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()

    plt.figure(figsize=(8, 5))
    plt.plot(val_precisions, label='Val Precision', marker='^')
    plt.plot(val_recalls, label='Val Recall', marker='x')
    plt.plot(val_f1s, label='Val F1', marker='|')
    plt.title("Precision, Recall, F1")
    plt.xlabel("Epoch")
    plt.ylabel("Score")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()

    trainingData = {
        "learningRate": learningRate,
        "epochs": (epoch, epochs),
        "treshold": threshold,
        "initial training loss": train_losses,
        "initial training acc": train_accuracies,
        "initial validation loss": val_losses,
        "initial validation acc": val_accuracies,
        "initial validation precision": val_precisions,
        "initial validation recall": val_recalls,
        "initial validation F1":val_f1s, 
        "best initial loss model": best_loss_model,
        "best initial f1 model": best_f1_model,
    }
    
    with open("loss_model.pkl", "wb") as f:
        pickle.dump(best_loss_model, f)
        print("Fiile saved")
        
    with open("f1_model.pkl", "wb") as f:
        pickle.dump(best_f1_model, f)
        print("Fiile saved")
        
    return trainingData

def train_loss_model(model:Ivan_NN, nFeatures:int, bestModelData:dict, training90_loader, test10_loader, epochs=20, learningRate = 0.00001, threshold:float = 0.5, device=None):
    # Use GPU if available
    device = device or ("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Loading best loss model State from epoch {bestModelData['epoch']}")
    finalModel = model(nFeatures)
    finalModel.load_state_dict(bestModelData["stateDict"])
    
    posWeight = torch.tensor(10, dtype=torch.float32).to("cpu")
    criterion = nn.BCEWithLogitsLoss(pos_weight=posWeight)
    optimizer = optim.Adam(finalModel.parameters() ,lr=learningRate, weight_decay=1e-4)

    train_losses = []
    train_accuracies = []

    for epoch in range(bestModelData["epoch"]):
        finalModel.train()
        running_loss = 0.0
        correct = 0
        total = 0

        for inputs, targets in training90_loader:
            optimizer.zero_grad()
            inputs, targets = inputs.to(device), targets.squeeze().to(device)

            # Forward pass
            outputs = finalModel(inputs).squeeze()
            loss = criterion(outputs, targets)
            # Backward and optimize
            # torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=.5)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()# * inputs.size(0)

            # preds = torch.round(torch.sigmoid(outputs))
            preds = (torch.sigmoid(outputs) >= threshold).float()


            correct += (preds == targets).sum().item()
            total += targets.size(0)

        train_loss = running_loss / total
        train_acc = correct / total
        print(f"Epoch [{epoch+1}/{epochs}]".ljust(16),
              f"Train Loss: {train_loss:.6f} Acc: {train_acc:.6f}")
        
        train_losses.append(train_loss)
        train_accuracies.append(train_acc)


    finalModel.eval()   

    with torch.no_grad():
        for inputs, targets in test10_loader:
            inputs, targets = inputs.to(device), targets.squeeze().to(device)
            outputs = finalModel(inputs).squeeze()
            loss = criterion(outputs, targets)
            test_loss = loss.item() # * inputs.size(0)

            # preds = torch.round(torch.sigmoid(outputs))
            preds = (torch.sigmoid(outputs) >= threshold).float()

            test_f1Score = f1_score(targets, preds)
            test_precision = precision_score(targets,preds)
            test_recall = recall_score(targets, preds)
            test_accuracy = accuracy_score(targets,preds)

            qwerty = [
                f"Loss: {test_loss:.6f}", 
                f"Accuracy: {test_accuracy:.2f}%", 
                f"Precision: {test_precision:.2f}", 
                f"Recall: {test_recall:.2f}", 
                f"F1 Score: {test_f1Score:.2f}",
            ]
            wwidth = 30
            print("Test Results".center(16).center(wwidth,"="))
            for line in qwerty:
                print(line.ljust(16).center(wwidth-2).center(wwidth,"|"))
            # print("Test", f"Loss: {test_loss:.6f}", f"Accuracy: {test_accuracy:.2f}%", f"Precision: {test_precision:.2f}", f"Recall: {test_recall:.2f}", f"F1 Score: {test_f1Score:.2f}%", sep="\n ")

    plt.figure(figsize=(8, 5))
    plt.plot(train_losses, label='Train Loss', marker='^')
    plt.title("Training Loss")
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()

    plt.figure(figsize=(8, 5))
    plt.plot(train_accuracies, label='Train Acc', marker='^')
    plt.title("Training Acc")
    plt.xlabel("Epoch")
    plt.ylabel("Accuracy")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()


    loss_Model = {
        "final training loss": train_losses,
        "final training acc": train_accuracies,
        "Test loss": test_loss,
        "Test acc": test_accuracy,
        "Test precision": test_precision,
        "Test recall": test_recall,
        "Test F1": test_f1Score,
        "final model": finalModel.state_dict(),
    }
    
    with open("Loss_Model.pkl", "wb") as f:
        pickle.dump(finalModel.state_dict(), f)
        print("Loss Model State Dict saved")

    
        
    return loss_Model

def train_f1_model(model:Ivan_NN, nFeatures:int, bestModelData:dict, trainVal_loader, test_loader, learningRate = 0.00001, threshold:float = 0.5, device=None):
    # Use GPU if available
    device = device or ("cuda" if torch.cuda.is_available() else "cpu")

    print(f"Loading best f1 model State from epoch {bestModelData['epoch']}")
    f1Model = model(nFeatures)
    f1Model.load_state_dict(bestModelData["stateDict"])
    posWeight = torch.tensor(10, dtype=torch.float32).to("cpu")
    criterion = nn.BCEWithLogitsLoss(pos_weight=posWeight)
    optimizer = optim.Adam(f1Model.parameters() ,lr=learningRate, weight_decay=1e-4)

    train_losses = []
    train_accuracies = []

    for epoch in range(bestModelData['epoch']):
        f1Model.train()
        running_loss = 0.0
        correct = 0
        total = 0

        for inputs, targets in trainVal_loader:
            optimizer.zero_grad()
            inputs, targets = inputs.to(device), targets.squeeze().to(device)

            # Forward pass
            outputs = f1Model(inputs).squeeze()
            loss = criterion(outputs, targets)
            # Backward and optimize
            # torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=.5)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()# * inputs.size(0)

            preds = torch.round(torch.sigmoid(outputs))
            preds = (torch.sigmoid(outputs) >= threshold).float()

            correct += (preds == targets).sum().item()
            total += targets.size(0)

        train_loss = running_loss / total
        train_acc = correct / total
        print(f"Epoch [{epoch+1}/{bestModelData['epoch']}]".ljust(16),
              f"Train Loss: {train_loss:.6f} Acc: {train_acc:.6f} | ")
        
        train_losses.append(train_loss)
        train_accuracies.append(train_acc)


    f1Model.eval()   

    with torch.no_grad():
        for inputs, targets in test_loader:
            inputs, targets = inputs.to(device), targets.squeeze().to(device)
            outputs = f1Model(inputs).squeeze()
            loss = criterion(outputs, targets)
            test_loss = loss.item() # * inputs.size(0)

            preds = torch.round(torch.sigmoid(outputs))
            preds = (torch.sigmoid(outputs) >= threshold).float()

            test_f1Score = f1_score(targets, preds)
            test_precision = precision_score(targets,preds)
            test_recall = recall_score(targets, preds)
            test_accuracy = accuracy_score(targets,preds)
            test_auc = roc_auc_score(targets, preds)

            qwerty = [
                f"Loss: {test_loss:.6f}", 
                f"Accuracy: {test_accuracy:.2f}%", 
                f"Precision: {test_precision:.2f}", 
                f"Recall: {test_recall:.2f}", 
                f"F1 Score: {test_f1Score:.2f}",
                f"AUC Score: {test_auc:.2f}",
            ]
            wwidth = 30
            print("Test Results".center(16).center(wwidth,"="))
            for line in qwerty:
                print(line.ljust(16).center(wwidth-2).center(wwidth,"|"))
            # print("Test", f"Loss: {test_loss:.6f}", f"Accuracy: {test_accuracy:.2f}%", f"Precision: {test_precision:.2f}", f"Recall: {test_recall:.2f}", f"F1 Score: {test_f1Score:.2f}%", sep="\n ")

    plt.figure(figsize=(8, 5))
    plt.plot(train_losses, label='Train Loss', marker='^')
    plt.title("Training Loss")
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()

    plt.figure(figsize=(8, 5))
    plt.plot(train_accuracies, label='Train Acc', marker='^')
    plt.title("Training Acc")
    plt.xlabel("Epoch")
    plt.ylabel("Accuracy")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()


    f1_Model = {
        "final training loss": train_losses,
        "final training acc": train_accuracies,
        "Test loss": test_loss,
        "Test acc": test_accuracy,
        "Test precision": test_precision,
        "Test recall": test_recall,
        "Test F1": test_f1Score,
        "final model": f1Model.state_dict(),
    }
    with open("f1_model.pkl", "wb") as f:
        pickle.dump(f1Model.state_dict(), f)
        print("F1 Model State Dict saved")
    
    return f1_Model



In [ ]:

#?  hyperparameter declarations
nInputs = len(training80_x.columns)
batchSize = 700
epoch = 1000
learningRate = 0.000001
treshold = 0.65

In [ ]:
 
#? main model training and validation loop
trainingData = train_model(Ivan_NN, nInputs, train80_loader, val10_loader, epochs=epoch, learningRate=learningRate, threshold=treshold, patience=30)


In [ ]:
import pickle
#? model training and test loop
best_loss_model = pickle.load(open("f1_model.pkl", "rb"))
testingData = train_f1_model(Ivan_NN, nInputs, best_loss_model, training90_loader, test10_loader,learningRate=learningRate, threshold=treshold)